In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_table('../Datasets/Diabetic Retinopathy/messidor_features.arff', sep=',', index_col=False, header=None)

In [3]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# AUTOML

In [11]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [12]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7569444444444444

In [13]:
import pickle
with open('./models/diabetic_retinopathy_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [4]:
import pickle
with open('./models/diabetic_retinopathy_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

automl.show_models()

FileNotFoundError: [Errno 2] No such file or directory: './models/diabetic_retinopathy_automl.pkl'

# RANDOM FOREST

In [30]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [31]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7083333333333334

# SVC

In [14]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42, probability=True)
params = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=5, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 2/5] END ....................C=0.1, gamma=1;, score=0.532 total time=   0.5s
[CV 5/5] END ....................C=0.1, gamma=1;, score=0.529 total time=   0.4s
[CV 3/5] END ....................C=0.1, gamma=1;, score=0.532 total time=   0.5s
[CV 4/5] END ....................C=0.1, gamma=1;, score=0.529 total time=   0.4s
[CV 1/5] END ....................C=0.1, gamma=1;, score=0.532 total time=   0.8s
[CV 2/5] END ..................C=0.1, gamma=0.1;, score=0.532 total time=   0.4s
[CV 1/5] END ..................C=0.1, gamma=0.1;, score=0.532 total time=   0.6s
[CV 3/5] END ..................C=0.1, gamma=0.1;, score=0.532 total time=   0.5s
[CV 4/5] END ..................C=0.1, gamma=0.1;, score=0.529 total time=   0.5s
[CV 2/5] END .................C=0.1, gamma=0.01;, score=0.532 total time=   0.4s
[CV 4/5] END .................C=0.1, gamma=0.01;, score=0.529 total time=   0.3s[CV 1/5] END .................C=0.1, gamma=0.01;

GridSearchCV(cv=5, estimator=SVC(probability=True, random_state=42), n_jobs=5,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             verbose=3)

In [15]:
y_hat = gridSearch.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.78125

In [16]:
import pickle
with open('./models/diabetic_retinopathy_gridsearch.pkl', 'wb') as f:
    pickle.dump(gridSearch, f)

In [5]:
import pickle
with open('./models/diabetic_retinopathy_gridsearch.pkl', 'rb') as f:
    model = pickle.load(f)

# LIME

In [5]:
import lime
import lime.lime_tabular
import tqdm 

In [6]:
continous_features = range(2, 18)
categorical_features = X_train.columns.drop(continous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['No Sign', 'Sign'], categorical_features=categorical_features, discretize_continuous=True)

# LIME Global

In [8]:
from lime import submodular_pick
import time

start_time = time.time()
exp1 = submodular_pick.SubmodularPick(explainer, X_test.values, model.predict_proba, sample_size=200, num_features=len(X_test.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
exp2 = submodular_pick.SubmodularPick(explainer, X_test.values, model.predict_proba, sample_size=200, num_features=len(X_test.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1127.284937620163 seconds ---
--- 960.6424679756165 seconds ---


In [25]:
def get_feature_imp(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff[0].values

In [26]:
feat_imp1 = get_feature_imp(exp1)
feat_imp2 = get_feature_imp(exp2)

In [27]:
feat_imp1

array([ 6.60797109e-02, -2.01083119e-02, -1.70482272e-02,  1.46429187e-03,
       -6.32766909e-03,  1.81562623e-03,  1.51379486e-04,  1.38767756e-03,
        4.24616947e-03, -3.71446214e-03, -1.40072010e-03, -4.16453260e-05,
        1.81055206e-02,  3.49338689e-04, -5.66418899e-04,  6.47858382e-04,
       -2.25833312e-03, -3.96235525e-07,  2.33639522e-04, -5.61476437e-02,
        2.74452802e-02,  9.59484416e-03,  7.31199534e-03,  5.89873102e-03,
        1.67762235e-02, -1.82812425e-04, -2.25993322e-03,  6.27170052e-05,
       -3.28158549e-05,  9.96064090e-04,  7.16737691e-04, -5.60762135e-02,
        1.00775398e-02,  1.23579753e-02,  6.14072807e-03, -7.26686503e-05,
       -1.41755661e-03,  3.41798980e-03,  2.92800781e-03, -3.80682543e-03,
        2.35768074e-03,  6.63435142e-04,  5.30753575e-04, -6.97393084e-04,
       -4.91630779e-03,  8.10373113e-04, -5.00869188e-04,  3.05297337e-01,
       -1.02423374e-01, -3.84658219e-02, -2.05085549e-02, -2.72911271e-02,
       -8.38089714e-05, -

In [28]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

In [29]:
i = global_identity(feat_imp1, feat_imp2)
i

0.0

In [30]:
def normal_fi(feat_imp):
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [31]:
normal_feat_imp = normal_fi(feat_imp1 + 1e-9)

In [32]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp*np.log(normal_feat_imp))/np.log(1/len(normal_feat_imp))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp*np.log(normal_feat_imp/(1/len(normal_feat_imp))))

In [33]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [34]:
Sg = calc_gini(normal_feat_imp)

In [35]:
Ser, Skl, Sg

(0.6373754076886349, 1.530097261344149, 0.00017553587383214452)

In [36]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [37]:
calc_alpha_fi(normal_feat_imp, 0.8)

0.7058823529411764

In [38]:
def get_feature_imp_all(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff

In [39]:
class1_feat_imp, class2_feat_imp = get_feature_imp_all(exp1)[0].values, get_feature_imp_all(exp1)[1].values
normal_class1_fi, normal_class2_fi = normal_fi(class1_feat_imp), normal_fi(class2_feat_imp)

In [40]:
np.linalg.norm(normal_class1_fi - normal_class2_fi, ord=2)

0.36539163769093935

# CIU

In [23]:
from ciu import determine_ciu
import tqdm
import metrics

In [24]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [25]:
feat_list = X_train.columns.tolist()

In [26]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], model.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [27]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

100%|██████████| 288/288 [09:33<00:00,  1.99s/it]


In [28]:
np.save('./explanations/diabetic_retinopathy_ciu1.npy', exp1)
np.save('./explanations/diabetic_retinopathy_ciu2.npy', exp2)

In [29]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [30]:
i, s, sb

((100.0, 0, 288), (0, 288, 82944, 0.0), (95, 288))

In [31]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [32]:
sim

0.9352786077097661

In [33]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(model, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

  2%|▏         | 7/288 [00:01<00:46,  6.04it/s]

100%|██████████| 288/288 [00:24<00:00, 11.76it/s]


In [34]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

-0.053242724289966445
0.013888888888888888
5.461805555555555
-0.08333333333333334
0.0
4.0


In [35]:
metrics.calc_trust_score(model, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

  6%|▌         | 17/288 [00:00<00:16, 16.15it/s]

100%|██████████| 288/288 [00:11<00:00, 24.32it/s]


0.3229166666666667

# RULEFIT

In [4]:
from skrules import SkopeRules
import metrics_rules
import time

In [11]:
# change dataset column names to x0, x1, x2, ...
X_train.columns = ['x'+str(i) for i in range(len(X_train.columns))]
X_test.columns = ['x'+str(i) for i in range(len(X_test.columns))]

In [12]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [13]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 20.400280237197876 seconds ---


In [14]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.07584309577941895 seconds ---


In [15]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 288, 288)
(30466, 288, 82944, 36.730806327160494)
(109, 288)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [16]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.8747086026218248


# RULEMATRIX

In [9]:
continuous_features = range(2, 18)
categorical_features = X_train.columns.drop(continuous_features).tolist()

In [10]:
import rulematrix
import time
import metrics_rules

In [11]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [12]:
surrogate = rulematrix.surrogate.rule_surrogate(
    model.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [13]:
test_x = X_test.to_numpy()

In [14]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [15]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.1182096004486084 seconds ---


In [16]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [17]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [18]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test)
print(sb)

(0.0, 288, 288)


(8556, 288, 82944, 10.315393518518519)
(119, 288)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [19]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [20]:
sim

1.330881702767274

# ANCHOR Global


In [6]:
from anchor import anchor_tabular
import anchor_utils
import tqdm

In [7]:
explainer = anchor_tabular.AnchorTabularExplainer(
    y_train.unique().tolist(),
    X_train.columns.tolist(),
    X_train.values
)

In [8]:
# Feature Importance using Anchor
def calc_fi(X_test, model, explainer):
    all_exps = []
    for i in tqdm.tqdm(range(len(X_test))):
        exp = explainer.explain_instance(X_test.values[i], model.predict, threshold=0.95)
        all_exps.append(exp.exp_map)
    fi = anchor_utils.greedy_pick_anchor(all_exps, X_test.values, k = len(X_test.columns))
    return fi
        

In [9]:
exp1 = calc_fi(X_test, model, explainer)
exp2 = calc_fi(X_test, model, explainer)

100%|██████████| 288/288 [03:26<00:00,  1.39it/s]


0 0.041666666666666664
1 0.07291666666666667
2 0.10069444444444445
3 0.125
4 0.14930555555555555
5 0.1701388888888889
6 0.1909722222222222
7 0.20833333333333334
8 0.2222222222222222
9 0.2361111111111111
10 0.25
11 0.2638888888888889
12 0.2777777777777778
13 0.2916666666666667
14 0.3020833333333333
15 0.3090277777777778
16 0.3159722222222222
17 0.3229166666666667
18 0.3298611111111111


100%|██████████| 288/288 [04:14<00:00,  1.13it/s]

0 0.041666666666666664
1 0.07291666666666667
2 0.10069444444444445
3 0.125
4 0.14930555555555555
5 0.1701388888888889
6 0.1909722222222222
7 0.21180555555555555
8 0.22569444444444445
9 0.23958333333333334
10 0.2534722222222222
11 0.2638888888888889
12 0.2743055555555556
13 0.2847222222222222
14 0.2951388888888889
15 0.3020833333333333
16 0.3090277777777778
17 0.3159722222222222
18 0.3229166666666667


In [10]:
def normal_fi(feat_imp):
    feat_imp = np.array(feat_imp) + 1e-9
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [11]:
normal_feat_imp1 = normal_fi(exp1)
normal_feat_imp2 = normal_fi(exp2)

In [12]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

i = global_identity(normal_feat_imp1, normal_feat_imp2)
i


0.0

In [13]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp1*np.log(normal_feat_imp1))/np.log(1/len(normal_feat_imp1))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp1*np.log(normal_feat_imp1/(1/len(normal_feat_imp1))))

In [14]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [15]:
Sg = calc_gini(normal_feat_imp1)

In [16]:
Ser, Skl, Sg

(0.9184919903354068, 0.23999536077070274, 0.0010646372628182186)

In [17]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [18]:
calc_alpha_fi(normal_feat_imp1, 0.8)

0.736842105263158